In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../Coding")
import Constants as con


In [2]:
df = pd.read_csv(f"{con.pathData}/ResultedVesselOnBerth.csv")

In [3]:
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])

# Step 1: Sort the dataframe by vehicle_id and start_time
df_sorted = df.sort_values(by=['IMO', 'start_time'])
# Step 2: Calculate the time difference between consecutive rows within each IMO group
df_sorted['time_diff'] = df_sorted['start_time'] - df_sorted.groupby('IMO')['end_time'].shift(1)

# Step 3: Identify the start of each trip based on the time difference
df_sorted['trip_start'] = (df_sorted['time_diff'] > pd.Timedelta(days=7)) | (df_sorted['time_diff'] < pd.Timedelta(days=0)) | df_sorted['time_diff'].isnull()

# Step 4: Create a trip ID for each group of consecutive rows
df_sorted['trip_id'] = df_sorted.groupby('IMO')['trip_start'].cumsum()

# Step 5: Create a trip sequence number within each trip
df_sorted['trip_sequence'] = df_sorted.groupby(['IMO', 'trip_id']).cumcount() + 1

# Step 6: Pivot the dataframe to create separate columns for each trip
df_pivot = df_sorted.pivot_table(index=['IMO', 'trip_id'], columns='trip_sequence', 
                                 values=['nearestPort', 'start_time', 'end_time'], aggfunc='first')

# Step 7: Flatten the multi-level column index
df_pivot.columns = ['_'.join(map(str, col)) for col in df_pivot.columns]

# Step 8: Calculate the time difference between consecutive cities
for i in range(2, len(df_pivot.columns) // 3 + 1):
    if f'start_time_{i}' in df_pivot.columns and f'end_time_{i-1}' in df_pivot.columns:
        df_pivot[f'time_diff_{i-1}_{i}'] = (df_pivot[f'start_time_{i}'] - df_pivot[f'end_time_{i-1}']).dt.total_seconds() / 3600

# Step 9: Drop unnecessary columns
#df_pivot.drop(columns=[f'city_{i}' for i in range(2, len(df_pivot.columns) // 3 + 1)], inplace=True)
#df_pivot.drop(columns=[f'start_time_{i}' for i in range(2, len(df_pivot.columns) // 3 + 1)], inplace=True)
#df_pivot.drop(columns=[f'end_time_{i}' for i in range(1, len(df_pivot.columns) // 3 + 1)], inplace=True)

In [4]:
df_pivot.to_csv(f"{con.pathData}/Routes2.csv")
df_sorted.to_csv(f"{con.pathData}/RoutesSorted2.csv")

In [7]:
df_pivot

end_time_1          end_time_2 end_time_3  \
IMO        trip_id                                                      
IMO7617905 1       2015-07-22 12:24:47                 NaT        NaT   
           2       2015-08-05 19:53:17                 NaT        NaT   
           3       2015-11-28 16:19:07                 NaT        NaT   
           4       2015-12-25 03:17:19                 NaT        NaT   
           5       2016-01-23 13:32:43                 NaT        NaT   
...                                ...                 ...        ...   
IMO9946245 2       2023-06-29 07:03:44 2023-09-19 06:46:37        NaT   
           3       2023-09-21 15:10:13                 NaT        NaT   
IMO9951135 1       2023-05-10 10:15:32                 NaT        NaT   
IMO9951147 1       2023-04-19 01:13:48                 NaT        NaT   
           2       2023-07-07 01:32:19                 NaT        NaT   

                   end_time_4 end_time_5 end_time_6 end_time_7 end_time_8  \
IMO        trip_id                                                          
IMO7617905 1              NaT        NaT        NaT        NaT        NaT   
           2              NaT        NaT        NaT        NaT        NaT   
           3              NaT        NaT        NaT        NaT        NaT   
           4              NaT        NaT        NaT        NaT        NaT   
           5              NaT        NaT        NaT        NaT        NaT   
...                       ...        ...        ...        ...        ...   
IMO9946245 2              NaT        NaT        NaT        NaT        NaT   
           3              NaT        NaT        NaT        NaT        NaT   
IMO9951135 1              NaT        NaT        NaT        NaT        NaT   
IMO9951147 1              NaT        NaT        NaT        NaT        NaT   
           2              NaT        NaT        NaT        NaT        NaT   

                   nearestPort_1 nearestPort_2  ... start_time_6 start_time_7  \
IMO        trip_id                              ...                             
IMO7617905 1              LA_APM           NaN  ...          NaT          NaT   
           2         LA_WBCT_ECT           NaN  ...          NaT          NaT   
           3         LA_WBCT_ECT           NaN  ...          NaT          NaT   
           4         LA_WBCT_ECT           NaN  ...          NaT          NaT   
           5         LA_WBCT_ECT           NaN  ...          NaT          NaT   
...                          ...           ...  ...          ...          ...   
IMO9946245 2        Philadelphia       Savanna  ...          NaT          NaT   
           3        Philadelphia           NaN  ...          NaT          NaT   
IMO9951135 1           LA_TRAPAC           NaN  ...          NaT          NaT   
IMO9951147 1           LA_TRAPAC           NaN  ...          NaT          NaT   
           2           LA_TRAPAC           NaN  ...          NaT          NaT   

                   start_time_8 time_diff_1_2 time_diff_2_3 time_diff_3_4  \
IMO        trip_id                                                          
IMO7617905 1                NaT           NaN           NaN           NaN   
           2                NaT           NaN           NaN           NaN   
           3                NaT           NaN           NaN           NaN   
           4                NaT           NaN           NaN           NaN   
           5                NaT           NaN           NaN           NaN   
...                         ...           ...           ...           ...   
IMO9946245 2                NaT   1959.015556           NaN           NaN   
           3                NaT           NaN           NaN           NaN   
IMO9951135 1                NaT           NaN           NaN           NaN   
IMO9951147 1                NaT           NaN           NaN           NaN   
           2                NaT           NaN           NaN           NaN   

                   t

In [13]:
df_tar = pd.read_csv(f'{con.pathOutput}/prediction_gr_5.csv')
df_pred = pd.read_csv(f'{con.pathOutput}/target_gr_5.csv')


In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def calculate_mape_for_sets(df_true, df_pred):
    assert df_true.shape == df_pred.shape, "Input DataFrames must have the same shape"
    epsilon = 0.01
    df_true_nonzero = df_true.replace(0, epsilon)

    # Calculate MAPE for each column
    #mape_per_column = ((df_true_nonzero - df_pred).abs() / df_true_nonzero).mean() * 100
    mape_per_column = ((df_true + epsilon - df_pred).abs() / (df_true + epsilon)).mean() * 100

    #mape_per_set = ((df_true - df_pred).abs() / df_true).mean() * 100
    return mape_per_column

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Ensure both arrays have the same shape
    if y_true.shape != y_pred.shape:
        # Reshape y_pred to match the shape of y_true
        y_pred = y_pred.reshape(y_true.shape)

    # Avoid division by zero
    mask = y_true != 0
    y_true, y_pred = y_true[mask], y_pred[mask]    
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 

def calculate_smape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    
    # Ensure that both inputs are numpy arrays of the same shape
    if y_true.shape != y_pred.shape:
        y_pred = y_pred.reshape(y_true.shape)

    # Calculate the absolute difference and the average of absolute values
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2

    # Avoid division by zero by replacing zeros in the denominator with a small value
    denominator[denominator == 0] = 1e-10  # Small constant to prevent division by zero

    # Calculate SMAPE for each element and then average over all elements
    smape = np.mean(numerator / denominator) * 100

    return smape

def saveError_n(fileName, terminal, model, description, actual, prediction, roundQty = 0):
    output = prediction.copy()
    y_true = np.nan_to_num(actual, nan=0.0)#actual.fillna(0)  # Fill NaNs with zero
    y_pred = np.nan_to_num(prediction, nan=0.0)#prediction.fillna(0) #y_pred.mean()  
    if roundQty == 1:
        output = output.round()
    err = {
        "roundQty" : [roundQty],
        "terminal": [f"{terminal}"],
        "model": [f"{model}"],
        "description": [f"{description}"],
        "mse": [mean_squared_error(y_true, y_pred)],
        "mae": [mean_absolute_error(y_true, y_pred)], 
        "RSquared": [r2_score(y_true, y_pred)],
        "mape": [mean_absolute_percentage_error(y_true, y_pred)], 
        "smape":[calculate_smape(y_true, y_pred)] 
    }

    pd.DataFrame(err).to_csv(f"{pathLogs}/{fileName}{roundQty}.csv")

def saveResults_n(fileName, actual, prediction):
    actual.to_csv(f'{pathOutput}/{fileName}_actual.csv', index=True)
    prediction.to_csv(f'{pathOutput}/{fileName}_prediction.csv', index=True)

###
def saveError_bt(fileName, model, description, actual, prediction, roundQty = 0):
    output = prediction.copy()
    #y_true = np.nan_to_num(actual, nan=0.0)
    y_true = actual.fillna(0)  # Fill NaNs with zero
    y_pred = np.nan_to_num(prediction, nan=0.0)#prediction.fillna(0) #y_pred.mean()  
    if roundQty == 1:
        output = output.round()
    error_df = pd.DataFrame()
    for col_idx in range(y_true.shape[1]):
        # Get the column name from y_test
        column_name = y_true.columns[col_idx] 
        y_true_col = y_true[column_name]
        y_pred_col = y_pred[:, col_idx]#y_pred.iloc[:, col_idx]
        err = {
            "roundQty" : roundQty,
            "terminal": f"{column_name}",
            "model": f"{model}",
            "description": f"{description}",
            "mse": mean_squared_error(y_true_col, y_pred_col),
            "mae": mean_absolute_error(y_true_col, y_pred_col), 
            "RSquared": r2_score(y_true_col, y_pred_col),
            "mape": mean_absolute_percentage_error(y_true_col, y_pred_col), 
            "smape":calculate_smape(y_true_col, y_pred_col) 
        }    
        error_df = error_df.append(err, ignore_index=True)  # Append the row to the DataFrame
    error_df.to_csv(f"{pathLogs}/{fileName}_error_br.csv")



In [14]:
from sklearn.metrics import r2_score
pathLogs = f"/home/gridsan/naristov/Modelling/Logs"
saveError_bt(f'STGNN', f'STGNN', f'several terminals', t_df(df_tar), t_df(df_pred), 0)

In [11]:
def t_df(df):
    df_t = df.T
    df_t.columns = df_t.iloc[0]  # Set the first row as column names

    # Step 2: Drop the first row
    df_t = df_t[1:]  # Keep all rows except the first one

    # Optional: Reset the index (if needed)
    df_t.reset_index(drop=True, inplace=True) 
    return df_t#.drop('Node',axis = 1)
t_df(df_tar)

Node,NY_Main,NY_Redhook,NY_LibertyBayonne,NY_LibertyNewYork,NY_Newark,NY_Maher,NY_APM,NY_WaitingArea,Boston_Main,Boston_Terminal,Boston_WaitingArea,Savanna_Main,Savanna_Terminal,Savanna_WaitingArea,Norfolk_Main,Norfolk_Terminal,Norfolk_WaitingArea,Baltimore_Main,Baltimore_Terminal,Baltimore_WaitingArea
0,3.445339,1.969851,2.618575,2.42467,2.293112,3.97137,2.806305,1.105891,1.37912,1.139584,1.041916,9.001441,1.217156,1.172351,2.578345,3.536646,10.597362,2.657808,3.411997,2.104856
1,3.469491,2.268634,2.865174,2.717899,2.520562,4.269483,2.99845,1.233583,1.488369,1.365787,1.167137,8.573973,1.248706,1.288317,2.679909,3.626118,9.786109,2.764374,3.507175,2.183356
2,3.31437,2.140712,2.716368,2.568616,2.309106,4.014861,2.748681,1.076072,1.312248,1.173701,1.012606,8.680462,1.117796,1.138211,2.482649,3.465454,10.024864,2.566298,3.357656,2.029864
3,3.660389,2.571865,3.159159,3.022193,2.612014,4.415468,3.1193,1.464308,1.706212,1.591805,1.396401,8.400251,1.518203,1.524241,2.8844,3.758138,9.905099,2.963644,3.645837,2.44135
4,3.470266,2.322603,2.967057,2.812394,2.369015,4.194376,2.933818,1.318921,1.551287,1.469454,1.257379,8.546658,1.320891,1.377102,2.744483,3.739187,9.668852,2.827963,3.62604,2.274086
5,3.602579,2.555052,3.139749,2.996463,2.636196,4.389417,3.104077,1.43574,1.676049,1.594122,1.359466,8.283447,1.436274,1.488672,2.854842,3.665162,9.972364,2.928309,3.556977,2.421581
6,3.724432,2.501672,3.10949,2.946541,2.654722,4.375315,3.168963,1.487279,1.766571,1.625401,1.446298,8.816769,1.589851,1.565251,2.939197,3.918233,10.246428,3.023268,3.800609,2.481441
7,3.549547,2.20421,2.814356,2.645821,2.40707,3.958053,2.920659,1.43415,1.676416,1.497339,1.372562,8.637367,1.457996,1.506287,2.815102,3.795309,10.036831,2.891154,3.686374,2.392105
8,3.595974,2.393095,2.983112,2.828888,2.525167,4.274824,2.962846,1.300237,1.556262,1.419266,1.260322,8.38487,1.395509,1.356516,2.642536,3.661129,9.476935,2.716495,3.549471,2.185252
9,3.568942,2.311883,2.952811,2.802633,2.459304,4.292821,2.952617,1.217612,1.446736,1.338505,1.14793,8.899204,1.242898,1.269194,2.670644,3.558789,10.48373,2.756565,3.435892,2.204813


In [2]:
userName = 'naristov'
pathOutput = f"/home/gridsan/{userName}/Modelling/Output"
#inp = pd.read_csv(f'{pathOutput}/INPUT_small.csv')

In [12]:
pd.DataFrame(inp.columns, columns=["Original Index"]).to_csv(f'{pathOutput}/INPUT_cols.csv')

In [4]:
importance = pd.read_csv(f'{pathOutput}/RF_importance.csv')  

In [5]:
importance[importance['Importance'] > 0.1]


,Feature,Importance,Output_Target
8720,Feature_8720,0.471103,Target_0
8721,Feature_8721,0.491297,Target_0
205880,Feature_8720,0.471103,Target_1
205881,Feature_8721,0.491297,Target_1
600203,Feature_8723,0.270860,Target_3
600204,Feature_8724,0.273357,Target_3
757043,Feature_165563,0.223045,Target_3
757044,Feature_165564,0.231907,Target_3
797363,Feature_8723,0.270860,Target_4
797364,Feature_8724,0.273357,Target_4
